# Bifacial Radiance Nespelem 

This journal supports the process of designing a solar panel configuration to appropriately represent ideal shading conditions for an Agripv setup in Nespelem, Washington.
The goal of this journal is to create a heat map which shows the amount of irradiance that reaches the ground for a certain layout.

Details:
*	Location: 48.167039, -118.976656; 
*	Desired area of initial analysis: 400-600 ft2 (37-55 m2)
*	Racking: Fixed-tilt panels
*	Panel size: 1.008 feet x 2.031 meters                                  
*	Analysis variations:
<ul> <li> a.	Analyzing and creating heat maps for each month. 
<li> b.	3 different azimuths per month </li> 
 

In [7]:
import bifacial_radiance
import os
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' /  'Nespelem_test2')
if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
resultsfolder = os.path.join(testfolder, 'results')

### General Parameters and Variables

In [9]:
lat = 48.167039
lon = -118.976656  #Lat and Long for Nespelem, WA

albedo = 0.31 # Average Albedo from Nespelem 
ft2m = 0.3048 # feet to meters conversion
    
# Loops
azimuths = np.array([180, 150, 210]) #cycling through 3 different azimuths
azimuth_strings = ['180','150','210']
azimuth_def=azimuths[0]


# Constants
clearance_height = 1.45 # Meters
xgap =  3 * ft2m
ygap = .02 # Meters
# D (meters)is a variable that represents the spacing between rows, not-considering the collector areas.
D = 2.8    
tilt = 25
x = 2.031
y = 1.008     
nMods = 7
nRows = 2
numpanels = 4
pitch = ((y * numpanels) * np.cos(np.radians(tilt)))+D
punder=((y * numpanels) * np.cos(np.radians(tilt))) #Length of area under module
moduletype = 'test-module' #Need to make sure this is right module
hpc = False
sim_general_name = 'nespelem'
print(pitch, punder)

6.454232997331772 3.6542329973317726


In [10]:
epwfile = r'EPWs\nespelem_wa_48.167039_-118.976656_psm3-tmy_60_tmy_TMY3formatV2.csv' 

<a id='step1'></a>

## 1. Create Geometry using gendaylit and view

In [5]:
demo = bifacial_radiance.RadianceObj(sim_general_name,str(testfolder))  
demo.setGround(albedo)
metdata = demo.readWeatherFile(epwfile, coerce_year=2020) # Name and path

path = C:\Users\jjones\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Nespelem_test2
Loading albedo, 1 value(s), 0.310 avg
1 nonzero albedo values.
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2020
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [12]:
timestamp = metdata.datetime.index(pd.to_datetime('2020-07-01 16:00:0 -8'))
demo.gendaylit(timestamp)  # Use this to simulate only one hour at a time.
azimuth_test = 180
pitch = ((y * numpanels) * np.cos(np.radians(tilt)))+D
sim_name = sim_general_name+'_az_'+str(round(azimuth_test,1))
module = demo.makeModule(name=moduletype, x=x, y=y,numpanels = numpanels, xgap=xgap, ygap=ygap)
sceneDict = {'tilt':tilt,'pitch':pitch,'clearance_height':clearance_height,'azimuth':azimuth_test,'nMods': nMods,'nRows': nRows}
scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
octfile = demo.makeOct(demo.getfilelist()) 


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten

Created nespelem.oct


In [14]:
#Un-comment the line below and run to view geometry
!rvu -vf views\front.vp -e .01 -pe 0.4 -vp 3.5 -20 22 nespelem.oct

## Recreate weather and geometry using genCumsky

In [11]:
# star date and end date for example for monthly data May 1 to May 31
demo = bifacial_radiance.RadianceObj(sim_general_name,str(testfolder))  
demo.setGround(albedo)
demo.readWeatherFile(epwfile) # Name and path
demo.genCumSky()

path = C:\Users\jjones\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Nespelem_test2
Loading albedo, 1 value(s), 0.310 avg
1 nonzero albedo values.
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loaded  EPWs\metdata_temp.csv
message: There were 4244 sun up hours in this climate file
Total Ibh/Lbh: 0.000000


'skies\\cumulative.rad'

## Main part of code. Creates Result files and heatmaps for each month and azimuth

In [21]:
# Parameters for scene size and sensor starts
sensorsx_fb = [1,1]
sensorsy_fb = [4,6*numpanels]
sensorsx_gb = [217,1] #217 for 10cm increase, originally 15
sensorsy_gb = [122,1] #122 for 10cm increase, originally 10
width = nMods * x +(nMods-1)*(xgap)+4
xinc = width / (sensorsx_gb[0] -1) 
xstart = -width/2
length = (4*y+(3*ygap))* np.cos(np.radians(tilt))*2+D+2
#ystart = -length/4
#ystart = length/1.328
ystart = length/1.1
#yinc = length/9
yinc = -length/(sensorsy_gb[0]-1) 

print(width,length,xstart,xinc,ystart,yinc)
print(length, pitch)

23.703400000000002 12.217222929107942 -11.851700000000001 0.10973796296296297 11.106566299189037 -0.10096878453808217
12.217222929107942 6.454232997331772


In [22]:
# Main part of code. 
module = demo.makeModule(name=moduletype, x=x, y=y, xgap=xgap, ygap=ygap)
# Prepare to run for loop of start and end times for May to October
starttimes =['2005-05-01 1:30:0', '2007-06-01 1:30:0', '2020-07-01 1:30:0', '2020-08-01 1:30:0', '2002-09-01 1:30:0', '1998-10-01 1:30:0']
endtimes = ['2005-05-31 23:30:0', '2007-06-30 23:30:0','2020-07-31 23:30:0', '2020-08-31 23:30:0', '2002-09-30 23:30:0', '1998-10-31 23:30:0']
months = ['May','Jun','Jul','Aug','Sep','Oct']
demo = bifacial_radiance.RadianceObj(sim_general_name,str(testfolder))  
demo.setGround(albedo)
for ti in range (0, len(starttimes)):
    starttime = pd.to_datetime(starttimes[ti])
    endtime = pd.to_datetime(endtimes[ti])
    month = months[ti]
    demo.readWeatherFile(epwfile, starttime=starttime, endtime=endtime) # Name and path
    demo.genCumSky()
    #Generate sky for each month and run emptyscan for each month
    sim_name = sim_general_name+'_'+month
    #Run an empty scan for each month to calculate total irradiance in each empty space
    sceneDict = {'tilt':0,'pitch':2,'clearance_height':0.005,'azimuth':180, 'nMods': 1, 'nRows': 1} 
    scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
    octfile = demo.makeOct(octname = demo.basename)  
    analysis = bifacial_radiance.AnalysisObj(octfile=octfile, name=sim_name)
    frontscan, backscan = analysis.moduleAnalysis(scene=scene,sensorsx = sensorsx_gb,sensorsy=sensorsy_gb)
    emptyscan = frontscan.copy() 
    emptyscan['xstart'] = xstart
    emptyscan['xinc'] = 0
    emptyscan['sx_xinc'] = xinc
    emptyscan['ystart'] = ystart
    emptyscan['yinc'] = yinc
    emptyscan['zstart'] = 0.05
    emptyscan['zinc'] = 0
    emptyscan['orient'] = '0 0 -1'
    emptybackscan = emptyscan.copy()
    emptybackscan['orient'] = '0 0 1'
    analysis.analysis(octfile, name=sim_name+'_EMPTYSCAN', frontscan=emptyscan, backscan=emptybackscan)

    for az in range (0, len(azimuths)):
        #Build scene and run raytracing for all 3 azimuths
        azimuth = azimuths[az]
        sim_name = sim_general_name+'_'+month+'_az_'+str(round(azimuth,1))
        sceneDict = {'tilt':tilt,'pitch':pitch,'clearance_height':clearance_height,'azimuth':azimuth,'nMods': nMods,'nRows': nRows}
        if azimuth == 180:
            scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
            octfile = demo.makeOct(demo.getfilelist())  
            analysis = bifacial_radiance.AnalysisObj(octfile=octfile, name=sim_name)
            frontscan, backscan = analysis.moduleAnalysis(scene=scene,sensorsx = sensorsx_gb,sensorsy=sensorsy_gb)
            groundscan = frontscan.copy() 
            groundscan['xstart'] = xstart
            groundscan['xinc'] = 0
            groundscan['sx_xinc'] = xinc
            groundscan['ystart'] = ystart
            groundscan['yinc'] = yinc
            groundscan['zstart'] = 0.05
            groundscan['zinc'] = 0
            groundscan['orient'] = '0 0 -1'
        sceneDict['azimuth'] = azimuth
        scene = demo.makeScene(moduletype=moduletype,sceneDict=sceneDict, radname = sim_name)
        octfile = demo.makeOct(demo.getfilelist())
        frontscan, backscan = analysis.moduleAnalysis(scene=scene, sensorsy=sensorsy_fb, sensorsx = sensorsx_fb)
        analysis.analysis(octfile, name=sim_name+'_Module_Analysis', frontscan=frontscan, backscan=backscan)
        analysis.analysis(octfile, name=sim_name+'_Ground_Analysis', frontscan=groundscan, backscan=backscan)
        #Read each azumith file for Wm2 Front
        azimuth_s = azimuth_strings[az]
        file = os.path.join(resultsfolder, 'irr_nespelem_'+month+'_az_'+azimuth_s+'_Ground_Analysis_Front.csv')
        df2 = pd.read_csv(file)
        Wm2 = np.asarray(df2['Wm2Front']) 
        Wms2 = np.reshape(np.ravel(Wm2, order='C'), (122,217), order='F')
        #Read each empty file for Wm2Front
        empty_scan = os.path.join(resultsfolder, 'irr_nespelem_'+month+'_EMPTYSCAN.csv')
        de = pd.read_csv(empty_scan)
        Wmempty = np.asarray(de['Wm2Front']) 
        Wms_empty = np.reshape(np.ravel(Wmempty, order='C'), (122,217), order='F')
        #Calculate Irradiance Facor and plot and save heat map of each month_azumith
        plt.figure(1)
        shading = Wms2/Wms_empty
        fig = plt.imshow(shading, cmap='hot', vmin=0.4, vmax=1, interpolation='none', aspect = 'equal',extent=[-11,11,0,12])
        plt.colorbar(label='Irr Factor')
        plt.title(month+'_Bifacial Irr Factor (az='+azimuth_s+'), in matrix form')
        fig.axes.get_yaxis().set_visible(False)
        fig.axes.get_xaxis().set_visible(False)
        plt.savefig(month+'_Bifacial_az_'+azimuth_s+'.png')
        plt.clf()
    #np.min(shading).min()
    #np.max(shading).max()
        
        






Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten

path = C:\Users\jjones\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Nespelem_test2
Loading albedo, 1 value(s), 0.310 avg
1 nonzero albedo values.
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2005
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loaded  EPWs\metdata_temp.csv
message: There were 448 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created nespelem.oct
Linescan in process: nespelem_May_EMPTYSCAN_Front
Linescan in process: nespelem_May_EMPTYSCAN_Back
Saved: results\irr_nespelem_May_EMPTYSCAN.csv
Created nespelem.oct
Created nespelem.oct
Linescan in process: n

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2020
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loaded  EPWs\metdata_temp.csv
message: There were 438 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created nespelem.oct
Linescan in process: nespelem_Aug_EMPTYSCAN_Front
Linescan in process: nespelem_Aug_EMPTYSCAN_Back
Saved: results\irr_nespelem_Aug_EMPTYSCAN.csv
Created nespelem.oct
Created nespelem.oct
Linescan in process: nespelem_Aug_az_180_Module_Analysis_Front
Linescan in process: nespelem_Aug_az_180_Module_Analysis_Back
Saved: results\irr_nespelem_Aug_az_180_Module_Analysis_Front.csv
Saved: results\irr_nespelem_Aug_az_180_Module_Analysis_Back.csv
Linescan in process: nespelem_Aug_az_180_Ground_Analysis_Front


<Figure size 432x288 with 0 Axes>

In [15]:
azimuths = np.array([180, 150, 210]) #cycling through 3 different azimuths
azimuth_strings = ['180','150','210']
starttimes =['2005-05-01 1:30:0', '2007-06-01 1:30:0', '2020-07-01 1:30:0', '2020-08-01 1:30:0', '2002-09-01 1:30:0', '1998-10-01 1:30:0']
endtimes = ['2005-05-31 23:30:0', '2007-06-30 23:30:0','2020-07-31 23:30:0', '2020-08-31 23:30:0', '2002-09-30 23:30:0', '1998-10-31 23:30:0']
months = ['May','Jun','Jul','Aug','Sep','Oct']
for ti in range (0, len(starttimes)):
    starttime = pd.to_datetime(starttimes[ti])
    endtime = pd.to_datetime(endtimes[ti])
    month = months[ti]
    for az in range (0, len(azimuths)):
        #Build scene and run raytracing for all 3 azimuths
        azimuth = azimuths[az]
        sim_name = sim_general_name+'_'+month+'_az_'+str(round(azimuth,1))
        azimuth_s = azimuth_strings[az]
        file = os.path.join(resultsfolder, 'irr_nespelem_'+month+'_az_'+azimuth_s+'_Ground_Analysis_Front.csv')
        df2 = pd.read_csv(file)
        Wm2 = np.asarray(df2['Wm2Front']) 
        Wms2 = np.reshape(np.ravel(Wm2, order='C'), (122,217), order='F')
        #Read each empty file for Wm2Front
        empty_scan = os.path.join(resultsfolder, 'irr_nespelem_'+month+'_EMPTYSCAN.csv')
        de = pd.read_csv(empty_scan)
        Wmempty = np.asarray(de['Wm2Front']) 
        Wms_empty = np.reshape(np.ravel(Wmempty, order='C'), (122,217), order='F')
        #Calculate Irradiance Facor and plot and save heat map of each month_azumith
        plt.figure(1)
        shading = Wms2/Wms_empty
        s_min = np.min(shading)
        s_ave = np.mean(shading)
        print('shading min for',month,azimuth,s_min)
        print('shading ave for',month,azimuth,s_ave)
        fig = plt.imshow(shading, cmap='hot', vmin=0.4, vmax=1, interpolation='none', aspect = 'equal', extent=[-11,11,0,12])
        plt.colorbar(label='Irr Factor')
        plt.title(month+'_Bifacial Irr Factor (az='+azimuth_s+'), in matrix form')
        fig.axes.get_yaxis().set_visible(False)
        fig.axes.get_xaxis().set_visible(False)
        plt.savefig(month+'_Bifacial_az_'+azimuth_s+'.png')
        plt.clf()
    #np.min(shading).min()
    #np.max(shading).max()

shading min for May 180 0.4788544507655635
shading ave for May 180 0.8938678720214434
shading min for May 150 0.5628103845066822
shading ave for May 150 0.9125419289024062
shading min for May 210 0.5985845940001592
shading ave for May 210 0.916854364480763
shading min for Jun 180 0.47218542348700177
shading ave for Jun 180 0.8975211656050384
shading min for Jun 150 0.5488223146768813
shading ave for Jun 150 0.9152671329039282
shading min for Jun 210 0.5819134412553595
shading ave for Jun 210 0.9173812012342806
shading min for Jul 180 0.4051650485436893
shading ave for Jul 180 0.8962836150212744
shading min for Jul 150 0.5228862413279289
shading ave for Jul 150 0.914433350662917
shading min for Jul 210 0.5289228363351879
shading ave for Jul 210 0.9213843883487236
shading min for Aug 180 0.4143870100522483
shading ave for Aug 180 0.8901442797392818
shading min for Aug 150 0.5456905781850188
shading ave for Aug 150 0.9113485131876402
shading min for Aug 210 0.5805778322570657
shading ave 

<Figure size 432x288 with 0 Axes>